In [29]:
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [78]:
from evaluation.evaluate_script import *
from loaders.loaders import *
from query.query_functions import *
from algorithm.grapwalk_functions_v1 import (node_excl_wiki_func, node_excl_owlthing_func, node_excl_yago_func, node_excl_extra,
                                             get_task_solution, clear_results, complex_partly_query_run, get_backward1,
                                             COUNT_construct_and_run_query_from_list)

import random
from collections import defaultdict
import numpy as np

In [89]:
lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)

In [90]:
sample_size = 4
seed = 42
i = 99
random.seed(seed)
record = lc_quad_train[i]
record_seed = random.sample(record["result_urlonly"], sample_size)
record_gold = list(set(record["result_urlonly"]) - set(record_seed))

In [91]:
record

{'_id': '1729',
 'corrected_question': 'What other destinations do airlines go to, whose flights also come to Glasgow?',
 'intermediary_question': 'What is the <destinations> of the <airlines> whose <target airport> is <Glasgow Airport>?',
 'sparql_query': 'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/ontology/targetAirport> <http://dbpedia.org/resource/Glasgow_Airport> . ?x <http://dbpedia.org/property/destinations> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Airline>}',
 'sparql_template_id': 305,
 'result': ['58',
  '42',
  '95',
  '102',
  '24',
  '96',
  'http://dbpedia.org/resource/Caribbean',
  'http://dbpedia.org/resource/Europe',
  'http://dbpedia.org/resource/North_America',
  '15',
  'http://dbpedia.org/resource/Aberdeen',
  'http://dbpedia.org/resource/Belfast',
  'http://dbpedia.org/resource/Glasgow',
  'http://dbpedia.org/resource/London',
  'http://dbpedia.org/resource/Manchester',
  '0',
  'http://dbpedia.org/resource

In [92]:
exclude_paths = [
    ["", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://www.w3.org/2004/02/skos/core#Concept"]
]
relation_excludes = ["http://xmlns.com/foaf/0.1/primaryTopic",
                     "http://dbpedia.org/property/isCitedBy",
                     "http://dbpedia.org/ontology/wikiPageWikiLink",
                     "http://dbpedia.org/ontology/wikiPageWikiLinkText",
                     "http://dbpedia.org/property/wikiPageUsesTemplate",
                     "http://dbpedia.org/ontology/abstract",
                     "http://www.w3.org/2000/01/rdf-schema#comment", ]

res_excludes_inside_point_relation = ["http://purl.org/linguistics/gold/hypernym",
                                      "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"]
res_excludes_inside_point_node = []
node_excludes_funcs = [node_excl_wiki_func, node_excl_owlthing_func, node_excl_yago_func, node_excl_extra]

In [93]:
result = get_task_solution(record_seed, exclude_paths, relation_excludes, node_excludes_funcs)

Forward Done!
Backward Done!


100%|███████████████████████████████████████████| 29/29 [05:09<00:00, 10.69s/it]

Backward Inside Point Done!
res_forward 2
res_backward 0
res_backward2 17


In [86]:
from algorithm.grapwalk_functions_v1 import run_grapwalk_function_v1
result, explanation = run_grapwalk_function_v1(record_seed, exclude_paths, relation_excludes,
                                               node_excludes_funcs, verbose=1)

Forward Done!
Backward Done!


100%|███████████████████████████████████████████| 24/24 [01:20<00:00,  3.36s/it]


Backward Inside Point Done!
res_forward 2
res_backward 0
res_backward2 47
record_seed ['http://dbpedia.org/resource/Caribbean', 'http://dbpedia.org/resource/Belfast', 'http://dbpedia.org/resource/Africa', 'http://dbpedia.org/resource/Aberdeen']
record_results:  {'http://dbpedia.org/resource/Brazil', 'http://dbpedia.org/resource/Queensland', 'http://dbpedia.org/resource/Singapore', 'http://dbpedia.org/resource/Germany', 'http://dbpedia.org/resource/Poland', 'http://dbpedia.org/resource/Hamburg', 'http://dbpedia.org/resource/Gujarat', 'http://dbpedia.org/resource/Israel', 'http://dbpedia.org/resource/Argentina', 'http://dbpedia.org/resource/England', 'http://dbpedia.org/resource/Uruguay', 'http://dbpedia.org/resource/Tehran', 'http://dbpedia.org/resource/Canada', 'http://dbpedia.org/resource/Pakistan', 'http://dbpedia.org/resource/Hungary', 'http://dbpedia.org/resource/Bristol', 'http://dbpedia.org/resource/Slovakia', 'http://dbpedia.org/resource/Algeria', 'http://dbpedia.org/resource/Ke

In [58]:
from query.query_functions import get_all_triplets_from_ro_excludes
a = "http://dbpedia.org/resource/Caribbean"
b ="http://purl.org/dc/terms/subject"
inside_points_raw = get_all_triplets_from_ro_excludes(a, b, excludes=exclude_paths)
print(inside_points_raw)

None


In [48]:
asd = [{'y': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Society_of_Advocates_in_Aberdeen'}}, {'y': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Aberdeen_City_Youth_Council'}}]

In [49]:
print([item["y"]["value"] for item in asd])

['http://dbpedia.org/resource/Society_of_Advocates_in_Aberdeen', 'http://dbpedia.org/resource/Aberdeen_City_Youth_Council']


In [ ]:
filtered_results = defaultdict(list)
for k, v in result.items():
    filtered_results[k] = clear_results(v)

backward2 = list(map(lambda x: [x, COUNT_construct_and_run_query_from_list(x)], filtered_results["backward2"]))
filtered_results["backward2"] = sorted(backward2, key=lambda x: x[1])

In [9]:
filtered_results["backward2"]

NameError: name 'filtered_results' is not defined

In [155]:
THRESHOLD = np.mean([item[1] for item in filtered_results["backward2"]])


In [159]:
record_results = complex_partly_query_run(filtered_results)


print("record_seed", record_seed)
print("record_gold: ", record_gold)

print("record_results: ", record_results)

for k in result.keys():
    print(len(result[k]), len(filtered_results[k]))

print("len record_results: ", len(record_results))

  1%|▎                                          | 2/266 [00:00<00:41,  6.41it/s]

6
6


  2%|▋                                          | 4/266 [00:00<00:41,  6.31it/s]

8
8


  2%|▉                                          | 6/266 [00:00<00:40,  6.36it/s]

...

262


100%|████████████████████████████████████████▊| 265/266 [00:49<00:00,  3.02it/s]

260


100%|█████████████████████████████████████████| 266/266 [00:49<00:00,  5.38it/s]

264
record_seed ['http://dbpedia.org/resource/Ayrton_Senna', 'http://dbpedia.org/resource/Alain_Prost', 'http://dbpedia.org/resource/Elio_de_Angelis', 'http://dbpedia.org/resource/Nelson_Piquet']
record_gold:  ['http://dbpedia.org/resource/Nigel_Mansell', 'http://dbpedia.org/resource/Patrick_Tambay', 'http://dbpedia.org/resource/David_Coulthard', 'http://dbpedia.org/resource/Heinz-Harald_Frentzen', 'http://dbpedia.org/resource/Riccardo_Patrese', 'http://dbpedia.org/resource/Jim_Clark', 'http://dbpedia.org/resource/Ralf_Schumacher', 'http://dbpedia.org/resource/Fernando_Alonso', 'http://dbpedia.org/resource/Michael_Schumacher', 'http://dbpedia.org/resource/Damon_Hill', 'http://dbpedia.org/resource/Didier_Pironi', 'http://dbpedia.org/resource/Niki_Lauda']
record_results:  {'http://dbpedia.org/resource/Alain_Prost', 'http://dbpedia.org/resource/Elio_de_Angelis', 'http://dbpedia.org/resource/Nelson_Piquet', 'http://dbpedia.org/resource/Ayrton_Senna'}
9 9
462 266
len record_results:  4


In [ ]:
last_part = [[item[1], item[0][1][2]] for item in filtered_results["backward2"]]

In [ ]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_rows', 500)
pdf = pd.DataFrame(last_part, columns = ["QueryResultCount", "EndNode"])
pdf_grouped = pdf.groupby("EndNode").agg({"QueryResultCount": lambda x: list(x)}).reset_index()
pdf_grouped["len"] = pdf_grouped["QueryResultCount"].apply(lambda x: len(x))
pdf_grouped["LastPart"] = pdf_grouped["EndNode"].apply(lambda x: x.split("/")[-1])
pdf_grouped["CorePart"] = pdf_grouped["EndNode"].apply(lambda x: "/".join(x.split("/")[:-1]))
pdf_grouped = pdf_grouped.sort_values("LastPart")

In [145]:
display(pdf_grouped)

,EndNode,QueryResultCount,len,LastPart,CorePart
0,43.73465 7.421333333333333,"[69, 71]",2,43.73465 7.421333333333333,
4,http://dbpedia.org/ontology/Agent,"[160025, 160276]",2,Agent,http://dbpedia.org/ontology
18,http://dbpedia.org/resource/Alain_Prost,"[17, 17, 20, 20, 32, 32, 35, 35, 39, 39, 39, 3...",20,Alain_Prost,http://dbpedia.org/resource
5,http://dbpedia.org/ontology/Athlete,"[23739, 23992]",2,Athlete,http://dbpedia.org/ontology
19,http://dbpedia.org/resource/Autodromo_Enzo_e_D...,"[27, 27, 28, 28]",4,Autodromo_Enzo_e_Dino_Ferrari,http://dbpedia.org/resource
20,http://dbpedia.org/resource/Autódromo_Internac...,"[10, 10]",2,Autódromo_Internacional_Nelson_Piquet,http://dbpedia.org/resource
21,http://dbpedia.org/resource/Ayrton_Senna,"[16, 16, 19, 19, 59, 59, 59, 59, 65, 65, 65, 65]",12,Ayrton_Senna,http://dbpedia.org/resource
22,http://dbpedia.org/resource/BMW_in_Formula_One,[33],1,BMW_in_Formula_One,http://dbpedia.org/resource
23,http://dbpedia.org/resource/Brabham,"[55, 55]",2,Brabham,http://dbpedia.org/resource
24,http://dbpedia.org/resource/Circuit_Gilles_Vil...,"[30, 32, 32, 36]",4,Circuit_Gilles_Villeneuve,http://dbpedia.org/resource


In [146]:
sum(pdf_grouped["len"].tolist())

284